In [1]:
import sys, os, itertools
import re, nltk
from nltk import pos_tag
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import sklearn, gensim
from sklearn.decomposition import PCA
from gensim.corpora import Dictionary
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
sys.path.append(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/src/')
os.chdir(os.path.expanduser("~")+'/Desktop/topic_modeling/fine_grained_topic_modeling_for_misinformation/data/')
from utils import preprocess_for_bow
from models.lda import LDAwrappers
from models.hdp import HDPwrapper
from models.gsdmm import MovieGroupProcessWrapper
from models.lftm import LFTMwrapper
from models.bertopic import BERTopicWrapper
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster import hierarchy as sch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import norm
from numpy import random
from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired
keybert = KeyBERTInspired()
mmr = MaximalMarginalRelevance(diversity=0.3)
chained_representation = [keybert, mmr]

In [2]:
setopics=["Virus Origin and bioweapon theory", "Virus contraction (who contracts the virus) and transmission (what transmits the virus)", 
                    "Incidence, Mortality and 'Casedemic'", "Prevention (Face Mask, Vitamin D, etc.)", 
                    "Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)", "Vaccines", "COVID-19 Denialism"]

# dataset 1

In [3]:
data1 = preprocess_for_bow('data.csv', preprocessing=False)
bertopic=BERTopicWrapper(data1['text'], data1['ids'], custom_vectorizer=True, representation_model=chained_representation)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [4]:
bertopic.set_topics(setopics, top_n=50, min_sim=0.35)

In [9]:
bertopic.sbert_cluster_set_topics()

In [10]:
bertopic.ctfidf_cluster_set_topics()

## Results from label inputs

In [15]:
print('this print displays the count of docs associated with the topic labels set')
print('')
for i in range(len(setopics)):
    print('topic:', setopics[i], 'yields ', len(bertopic.topics_set_embeddings[i]['topic_idxs']),
            'distributed over', len(bertopic.topics_set_embeddings[i]['topic_labels']),'original bertopics')

this print displays the count of docs associated with the topic labels set

topic: Virus Origin and bioweapon theory yields  51 distributed over 15 original bertopics
topic: Virus contraction (who contracts the virus) and transmission (what transmits the virus) yields  23 distributed over 14 original bertopics
topic: Incidence, Mortality and 'Casedemic' yields  9 distributed over 46 original bertopics
topic: Prevention (Face Mask, Vitamin D, etc.) yields  24 distributed over 5 original bertopics
topic: Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) yields  77 distributed over 6 original bertopics
topic: Vaccines yields  12 distributed over 50 original bertopics
topic: COVID-19 Denialism yields  13 distributed over 50 original bertopics


### reclustering corpus on centroids extracted from docs above (SBERT)

In [16]:
np.unique(bertopic.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(setopics[i],'->',np.unique(bertopic.sbert_clusters, return_counts=True)[1][i])

Virus Origin and bioweapon theory -> 1921
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> 5233
Incidence, Mortality and 'Casedemic' -> 1378
Prevention (Face Mask, Vitamin D, etc.) -> 734
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> 3892
Vaccines -> 1744
COVID-19 Denialism -> 15289


In [18]:
# display outlier examples from 25% to 10%
top_to_remove = 0.25
bottom_to_remove = 0.35
outliersMinP=bertopic.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data1['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 0

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 157
example: I’ve decided to “identify” as Chinese. Coke will like me, Delta will agree with my “values” and I’ll probably get shoes from Nike & tickets to @MLB games. Ain’t America great?

Incidence, Mortality and 'Casedemic'
total 184
example: The coronavirus mortality rate has been shown to be 0.03%

Prevention (Face Mask, Vitamin D, etc.)
total 0

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 83
example: FDA is the global authority on food standards & it permitting 4% of cockroaches in 100gm bar of chocolate.

Vaccines
total 168
example: Experimental vaccines on children who have an approximate 0% chance of dying from the virus for which they are being vaccinated against is a type of child abuse. ZERO long term trials conducted. I just cannot understand parents that would allow their kids to be guinea pigs.

COVID-19 Denial

In [19]:
# display outlier examples from 25% to 10%
top_to_remove = 0.8
bottom_to_remove = 1
outliersMinP=bertopic.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data1['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 55
example: Sir David Attenborough criticised Israel for its conflict with Palestine.

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 784
example: A famous photograph of the Loch Ness Monster was actually just a whale penis.

Incidence, Mortality and 'Casedemic'
total 33
example: Suicides have virtually doubled since lockdown.

Prevention (Face Mask, Vitamin D, etc.)
total 72
example: Taking a person's temperature with an infrared thermometer near their head risks damaging the pineal gland, which is located in the brain.

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 425
example: Press release about ""Restricted Movement Order"" issued by India's PMO

Vaccines
total 431
example: Vaccines contain toxic ingredients

COVID-19 Denialism
total 2046
example: Facebook is shut down its Messenger service in December 2020



In [20]:
# display outlier examples from 25% to 10%
top_to_remove = 0.05
bottom_to_remove = 0.1
outliersMinP=bertopic.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data1['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 0

Virus contraction (who contracts the virus) and transmission (what transmits the virus)
total 390
example: The set of photos show a doctor who stole kidney of a COVID-19 patient at Garg Hospital, Muzaffarnagar, UP.

Incidence, Mortality and 'Casedemic'
total 6
example: A woman found human ashes in a $2 blue heart ornament.

Prevention (Face Mask, Vitamin D, etc.)
total 0

Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc)
total 17
example: Anti-inflammatories should be used to fight Covid-19.

Vaccines
total 60
example: Singapore halts the rollout of influenza vaccination due to deaths in South Korea

COVID-19 Denialism
total 1120
example: I think my favorite people are the ones who simultaneously claim that COVID was intentionally unleashed by governments for population control and then they say shit like “THE FLU/CAR ACCIDENTS KILL MORE PEOPLE THAN COVID!!! BAN CARS!!!!” Super effective population control



# dataset 2

In [21]:
data2 = preprocess_for_bow('claimreviews.csv', preprocessing=False)
bertopic2=BERTopicWrapper(data2['text'], data2['ids'], custom_vectorizer=True, representation_model=chained_representation)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [28]:
bertopic2.set_topics(setopics, top_n=50, min_sim=0.35)

In [29]:
bertopic2.sbert_cluster_set_topics()

In [30]:
bertopic2.ctfidf_cluster_set_topics()

In [31]:
print('this print displays the count of docs associated with the topic labels set')
print('')
for i in range(len(setopics)):
    print('topic:', setopics[i], 'yields ', len(bertopic2.topics_set_embeddings[i]['topic_idxs']),
            'distributed over', len(bertopic2.topics_set_embeddings[i]['topic_labels']),'original bertopics')

this print displays the count of docs associated with the topic labels set



TypeError: object of type 'int' has no len()

In [ ]:
np.unique(bertopic2.sbert_clusters, return_counts=True)
for i in range(len(setopics)):
    print(setopics[i],'->',np.unique(bertopic2.sbert_clusters, return_counts=True)[1][i])

Virus Origin and bioweapon theory -> 2800
Virus contraction (who contracts the virus) and transmission (what transmits the virus) -> 41
Incidence, Mortality and 'Casedemic' -> 6400
Prevention (Face Mask, Vitamin D, etc.) -> 6032
Treatment (Ivermectim, chloroquine and hydroxychloroquine, etc) -> 647
Vaccines -> 1616
COVID-19 Denialism -> 459


In [ ]:
# display outlier examples from 25% to 10%
top_to_remove = 0.4
bottom_to_remove = 0.5
outliersMinP=bertopic2.get_sbert_z_outliers(norm.ppf(bottom_to_remove/2))
outliersMaxP=bertopic2.get_sbert_z_outliers(norm.ppf(top_to_remove/2))
for i in range(len(setopics)):
    print(setopics[i])
    n=len(list(set(outliersMinP[i])-set(outliersMaxP[i])))
    print('total', n)
    if n>0:
        print('example:',data2['text'][list(set(outliersMinP[i])-set(outliersMaxP[i]))[random.randint(n)]])
    print('')

Virus Origin and bioweapon theory
total 210
example: Fake Messages Claim Shaheen Bagh Protester Tested Positive For CoronavirusBOOM reached out to Dr. Ram Manohar Lohia hospital and Safdarjung hospital in Delhi. Both denied receiving any Coronavirus patient from Shaheen Bagh.Viral tweets claiming a woman protester at Delhi's Shaheen Bagh tested positive for Coronavirus yet refused treatment and returned to the protest venue, are false.BOOM contacted one of the organisers of the anti-Citizenship Amendment Act protests at Shaheen Bagh and two hospitals in Delhi that are designated to treat Coronavirus patients. The organiser rubbished the message as fake meanwhile the two hospitals said they have not received any Coronavirus patient from Shaheen Bagh so far.The screenshot of a tweet that is being shared on WhatsApp reads, ""Big Breaking: Nazma begum ( 43 ) sitting in # ShaheenBaghProtest was not feeling well. Doctors found her coronavirus positive. She denied the treatment and again went